# importing the libararies 

In [9]:
import numpy as np 
import pandas as pd 
import random 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import math 
import pandas_datareader as data_reader 

from tqdm import tqdm_notebook , tqdm 
from collections import deque 

# building the AI trader network

In [10]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
            
        

# data set preprocessing 

## sigmoid 

In [11]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

## price format function 

In [12]:
def stock_price_format(n):
    if n < 0 :
        return '- $ {:2f}'.format(abs(n))
    else :
        return '$ {:2f}'.format(abs(n))

# dataset loader 

In [13]:
def dataset_loader(stock_name):
    
    dataset = data_reader.DataReader(stock_name , data_source = 'yahoo')
    
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    return close  

In [14]:

# this is for example purpose 


dataset_for_example = data_reader.DataReader('AAPL' , data_source = 'yahoo')
dataset_for_example



,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-08-24,108.800003,92.000000,94.870003,103.120003,162206300.0,95.294647
2015-08-25,111.110001,103.500000,111.110001,103.739998,103601600.0,95.867607
2015-08-26,109.889999,105.050003,107.089996,109.690002,96774600.0,101.366074
2015-08-27,113.239998,110.019997,112.230003,112.919998,84616100.0,104.350983
2015-08-28,113.309998,111.540001,112.169998,113.290001,53164400.0,104.692871
...,...,...,...,...,...,...
2020-08-14,460.000000,452.179993,459.320007,459.630005,41391300.0,459.630005
2020-08-17,464.350006,455.850006,464.250000,458.429993,29890400.0,458.429993
2020-08-18,464.000000,456.029999,457.410004,462.250000,26408400.0,462.250000


# state creator 

In [15]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

# loading a dataset 

In [16]:
stock_name = 'BUTTERFLY.BO'
data = dataset_loader(stock_name)

data 

Date
2015-08-24    148.500000
2015-08-25    145.699997
2015-08-26    144.300003
2015-08-27    151.300003
2015-08-28    150.199997
                 ...    
2020-08-17    137.350006
2020-08-18    161.550003
2020-08-19    163.750000
2020-08-20    173.199997
2020-08-21    207.800003
Name: Close, Length: 1228, dtype: float64

# Training the AI trader

## setting the hyper parameters 

In [17]:
window_size = 10 
episodes = 1000

batch_size = 32
data_samples = len(data) - 1 


## defining the trader model 

In [18]:
trader = AI_Trader(window_size)

In [19]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# training loop 

In [ ]:

 for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stock_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|                                                                                         | 0/1227 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 148.500000
AI Trader sold:  $ 145.699997  Profit: - $ 2.800003
AI Trader bought:  $ 144.300003
AI Trader bought:  $ 151.300003
AI Trader bought:  $ 150.199997
AI Trader bought:  $ 150.300003
AI Trader bought:  $ 145.000000
AI Trader sold:  $ 146.199997  Profit: $ 1.899994
AI Trader bought:  $ 147.500000
AI Trader bought:  $ 149.699997
AI Trader sold:  $ 146.699997  Profit: - $ 4.600006
AI Trader sold:  $ 150.699997  Profit: $ 0.500000
AI Trader bought:  $ 147.800003
AI Trader sold:  $ 148.699997  Profit: - $ 1.600006
AI Trader bought:  $ 152.899994
AI Trader sold:  $ 150.600006  Profit: $ 5.600006
AI Trader sold:  $ 150.500000  Profit: $ 3.000000
AI Trader sold:  $ 152.000000  Profit: $ 2.300003
AI Trader bought:  $ 154.800003
AI Trader sold:  $ 154.600006  Profit: $ 6.800003
AI Trader bought:  $ 153.500000
AI Trader sold:  $ 153.699997  Profit: $ 0.800003
AI Trader bought:  $ 154.699997
AI Trader bought:  $ 152.800003
AI Trader bought:  $ 163.39999

  3%|██▏                                                                             | 33/1227 [00:09<05:46,  3.44it/s]

AI Trader sold:  $ 165.199997  Profit: $ 10.399994


  3%|██▎                                                                           | 36/1227 [00:32<1:41:08,  5.10s/it]

...

 36%|███████████████████████████▋                                                 | 441/1227 [36:30<1:01:06,  4.66s/it]

AI Trader sold:  $ 180.350006  Profit: $ 16.900009


 36%|███████████████████████████▋                                                 | 442/1227 [36:35<1:01:24,  4.69s/it]

AI Trader sold:  $ 189.149994  Profit: $ 26.750000


 38%|██████████████████████████████▎                                                | 471/1227 [38:53<58:44,  4.66s/it]